In [1]:
import os
import speech_recognition as sr
import re

# Function to transcribe audio using Google Web Speech API
def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)  # Read the entire audio file

    try:
        text = recognizer.recognize_google(audio)
        return text.lower()  # Convert to lowercase for consistency
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError:
        return "Could not request results; check your network connection"

# Function to convert words to numbers in text
def words_to_numbers(text):
    word_to_num = {
        'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4,
        'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9,
        'ten': 10, 'eleven': 11, 'twelve': 12, 'thirteen': 13, 'fourteen': 14,
        'fifteen': 15, 'sixteen': 16, 'seventeen': 17, 'eighteen': 18, 'nineteen': 19,
        'twenty': 20, 'thirty': 30, 'forty': 40, 'fifty': 50, 'sixty': 60,
        'seventy': 70, 'eighty': 80, 'ninety': 90
    }

    text = text.lower()
    for word, num in word_to_num.items():
        text = text.replace(word, str(num))
    return text

# Function to extract and classify instructions from the transcription
def extract_and_classify_instructions(transcription):
    instructions = []

    # Convert words to numbers
    transcription = words_to_numbers(transcription)
    
    # Define regex pattern for instructions
    pattern = re.compile(r'(\d+)\s*(meters?|m|centimeters?|cm)?\s*(forward|up|backward|down|left|right|fly)\b|'
                         r'(forward|up|backward|down|left|right|fly)\s*(\d+)\s*(meters?|m|centimeters?|cm)?\b')
    
    matches = pattern.findall(transcription)
    
    for match in matches:
        if match[0]:  # matches the first group (direct pattern)
            distance, unit, direction = match[0], match[1], match[2]
        else:  # matches the second group (reverse pattern)
            direction, distance, unit = match[3], match[4], match[5]
        
        distance = int(distance) if distance.isdigit() else None
        unit = unit if unit else 'meters'
        
        if direction:
            direction = direction.lower()
            action = "Fly" if "fly" in direction else "Move"
            if direction in ['forward', 'up', 'backward', 'down', 'left', 'right', 'fly']:
                instructions.append({
                    "instruction": f"{action} {direction} by {distance} {unit}",
                    "details": {
                        "distance": distance,
                        "unit": unit,
                        "direction": direction
                    }
                })
    
    if not instructions:
        instructions.append("No valid instructions found.")
    
    return instructions

# Directory path where audio files are located
directory_path = r"C:\Users\tanya\OneDrive\Desktop\pytrch\nkb\New2"

# Open a file to write the results
with open("output.txt", "w") as file:
    # Process each audio file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".wav"):  # Adjust the extension if your files are in a different format
            audio_path = os.path.join(directory_path, filename)
            file.write(f"Processing file: {filename}\n")

            # Step 1: Transcribe the audio
            transcription = transcribe_audio(audio_path)
            file.write(f"Transcription: {transcription}\n")

            # Step 2: Extract and classify instructions
            instructions = extract_and_classify_instructions(transcription)
            
            if isinstance(instructions, list) and instructions:
                for i, instruction in enumerate(instructions, start=1):
                    if isinstance(instruction, dict):
                        file.write(f"Instruction {i}: {instruction['instruction']}\n")
                        file.write("Details:\n")
                        file.write(f"Distance: {instruction['details']['distance']}, "
                              f"Unit: {instruction['details']['unit']}, "
                              f"Direction: {instruction['details']['direction']}\n")
                        file.write("\n")
                    else:
                        file.write(f"{instruction}\n")
            else:
                file.write("No instructions found.\n")
            file.write("\n")  # Print a blank line for clarity between different files
